In [1]:
import sys

In [6]:
x1 = 5
t1 = 257

In [7]:
x2 = 5
t2 = 257

In [9]:
t3 = t2

In [10]:
x1 is x2, t1 is t2, t3 is t2

(True, False, True)

In [11]:
t1 = 257
t2 = 257

t1 is t2

False

In [12]:
s1 = "name"

In [13]:
s2 = "name"

In [14]:
s1 is s2

True

In [15]:
s3 = "!name"

In [16]:
s4 = "!name"

In [17]:
s3 is s4

False

In [20]:
s = "na"
s3 = s + "me"
s4 = s + "me"

s3 is s4

False

In [23]:
sys.getrefcount(0), sys.getrefcount(2 ** 8 + 1)

(5544, 2)

In [24]:
lst = [1, 2, 3]

sys.getrefcount(lst)

2

In [29]:
lst1 = [1, 2, 3]
lst1.append(lst1)

lst1

[1, 2, 3, [...]]

In [30]:
sys.getrefcount(lst1)

10

In [32]:
lst1, lst1[3]

([1, 2, 3, [...]], [1, 2, 3, [...]])

In [39]:
lst = [1, 2, 3]
print("init", sys.getrefcount(lst))

lst2 = lst
print("lst2", sys.getrefcount(lst))

lst3 = lst
print("lst3", sys.getrefcount(lst))

tp = (4, 5, lst)
print("tp", sys.getrefcount(lst))

del lst2, lst3, tp
print("del", sys.getrefcount(lst))

def count_refs(obj):
    print("count_refs", sys.getrefcount(obj))

count_refs(lst)
print("call", sys.getrefcount(lst))

init 2
lst2 3
lst3 4
tp 5
del 2
count_refs 4
call 2


In [53]:
import ctypes
import gc

In [52]:
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]


lst1 = [1, 2, 3]
lst1.append(lst1)

lst_id = id(lst1)

del lst1

#print("sys", sys.getrefcount(lst1))
print("cty", PyObject.from_address(lst_id).refcnt)

cty 1


In [56]:
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]


gc.disable()

lst1 = [1, 2, 3]
lst1.append(lst1)

lst_id = id(lst1)

del lst1

gc.collect()
gc.enable()

#print("sys", sys.getrefcount(lst1))
print("cty", PyObject.from_address(lst_id).refcnt)

cty 0


In [55]:
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]


gc.disable()

lst1 = [1, 2, 3]
lst1.append(lst1)

lst_id = id(lst1)

del lst1

gc.collect()
gc.enable()

#print("sys", sys.getrefcount(lst1))
print("cty", PyObject.from_address(lst_id).refcnt)

cty 22


In [59]:
len(gc.get_objects()), [len(gc.get_objects(generation=i)) for i in range(3)]

(180667, [423, 2338, 177918])

In [60]:
import weakref

In [66]:
class A:
    pass


obj = A()
print("init", sys.getrefcount(obj))

weak = weakref.ref(obj)
print("weak", sys.getrefcount(obj))

obj2 = weak()
print("obj2", sys.getrefcount(obj))

del obj, obj2

obj3 = weak()
print("weak", sys.getrefcount(obj3), obj3)

init 2
weak 2
obj2 3
weak 210242 None


In [ ]:
class Engine:
    def __init__(self, car):
        self.car = weakref.ref(car)


class Car:
    def __init__(self):
        self.engine = Engine(self)


car = Car()

In [84]:
N = 10_000_000

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


class PointSlots:
    __slots__ = ("x", "y")
    def __init__(self, x, y):
        self.x = x
        self.y = y
        

def process_list(lst):
    for obj in lst:
        z = obj.x + obj.y
        obj.x += obj.y
        obj.y = z

In [85]:
%%time

points = [Point(5, 1) for _ in range(N)]

CPU times: user 8.94 s, sys: 634 ms, total: 9.57 s
Wall time: 10.7 s


In [86]:
%%time

slot_points = [PointSlots(5, 1) for _ in range(N)]

CPU times: user 5.6 s, sys: 245 ms, total: 5.85 s
Wall time: 6.4 s


In [89]:
%%time

process_list(points)

CPU times: user 1.96 s, sys: 30 ms, total: 1.99 s
Wall time: 2.24 s


In [90]:
%%time

process_list(slot_points)

CPU times: user 1.52 s, sys: 15.9 ms, total: 1.54 s
Wall time: 1.63 s


In [94]:
sys.getsizeof(""), sys.getsizeof("1"), sys.getsizeof("12"), sys.getsizeof("Щ"), sys.getsizeof("ЩЩ")

(49, 50, 51, 76, 78)

In [96]:
sys.getsizeof([]), sys.getsizeof([1]), sys.getsizeof([1, 2])

(56, 64, 72)

In [98]:
sys.getsizeof({}), sys.getsizeof({1: 11})

(64, 232)

In [99]:
p = Point(10, 20)

In [102]:
p.__dict__, sys.getsizeof(p.__dict__), sys.getsizeof(p)

({'x': 10, 'y': 20}, 104, 48)

In [103]:
s = PointSlots(10, 20)

sys.getsizeof(s)

48

In [104]:
def calc_sum(a, b):
    return a + b


def calc_percent(a, p):
    b = calc_sum(a, a * p)
    return b


calc_percent(10, 0.1)

11.0

In [105]:
import cProfile, pstats, io

In [106]:
pr = cProfile.Profile()
pr.enable()

process_list(points)

pr.disable()


s = io.StringIO()
sortby = "cumulative"
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()

print(s.getvalue())

         39 function calls in 2.994 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    2.994    1.497 /Users/g.kandaurov/projects/deep_python_autumn_2022/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3342(run_code)
        2    0.000    0.000    2.994    1.497 {built-in method builtins.exec}
        1    0.000    0.000    2.994    2.994 /var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_4086/1586059520.py:1(<module>)
        1    2.994    2.994    2.994    2.994 /var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_4086/2475959611.py:16(process_list)
        2    0.000    0.000    0.000    0.000 /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/codeop.py:149(__call__)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000    0.000    0.000    0.000 /Library/Frameworks/Python.framework/Versions/3.10/l

In [110]:
pr = cProfile.Profile()
pr.enable()

calc_percent(10, 0.1)
calc_percent(100, 0.1)
calc_percent(100, 0.001)


pr.disable()


s = io.StringIO()
sortby = "cumulative"
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()

print(s.getvalue())

         80 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/codeop.py:149(__call__)
        4    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        4    0.000    0.000    0.000    0.000 /Users/g.kandaurov/projects/deep_python_autumn_2022/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3342(run_code)
        4    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        4    0.000    0.000    0.000    0.000 /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/contextlib.py:279(helper)
        8    0.000    0.000    0.000    0.000 {built-in method builtins.next}
        4    0.000    0.000    0.000    0.000 /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/contextlib.py:130(__enter__)
        4    0.000